## Module loading

In [4]:
from importlib import reload

import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

from lib.data import tools
from lib.data.tools import get_dataloader
from lib.train.runners import PytorchRunner
from lib.train.metrics import IoUMetricMeter
from models.modelInterface import BDD100kModel
import lib.simulation.env
reload(lib.simulation.env)
from lib.simulation.env import (get_image_paths, get_label_paths, get_model_additional_configs, get_transforms)

## Set parameters

In [5]:
all_conditions = [
        ["clear", "residential", "daytime"] ,
        ["clear", "residential", "night"] ,
        ["clear", "residential", "dawn/dusk"] ,
        ["clear", "city street", "daytime"] ,
        ["clear", "city street", "night"] ,
        ["clear", "city street", "dawn/dusk"] ,
        ["clear", "highway", "daytime"] ,
        ["clear", "highway", "night"] ,
        ["clear", "highway", "dawn/dusk"] ,
        ["overcast", "residential", "daytime"] ,
        ["overcast", "city street", "daytime"] ,
        ["overcast", "city street", "dawn/dusk"] ,
        ["overcast", "highway", "daytime"] ,
        ["overcast", "highway", "dawn/dusk"] ,
        ["undefined", "residential", "daytime"] ,
        ["undefined", "city street", "daytime"] ,
        ["undefined", "city street", "night"] ,
        ["undefined", "city street", "dawn/dusk"] ,
        ["undefined", "highway", "daytime"] ,
        ["partly cloudy", "residential", "daytime"] ,
        ["partly cloudy", "city street", "daytime"] ,
        ["partly cloudy", "city street", "dawn/dusk"] ,
        ["partly cloudy", "highway", "daytime"] ,
        ["partly cloudy", "highway", "dawn/dusk"] ,
        ["rainy", "residential", "daytime"] ,
        ["rainy", "city street", "daytime"] ,
        ["rainy", "city street", "night"] ,
        ["rainy", "city street", "dawn/dusk"] ,
        ["rainy", "highway", "daytime"] ,
        ["rainy", "highway", "night"] ,
        ["snowy", "residential", "daytime"] ,
        ["snowy", "residential", "night"] ,
        ["snowy", "city street", "daytime"] ,
        ["snowy", "city street", "night"] ,
        ["snowy", "city street", "dawn/dusk"] ,
        ["snowy", "highway", "daytime"] ,
        ["snowy", "highway", "night"]
    ]

In [6]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pkg_name = "10k" # 100k or 10k
output_size = (512,1024)
task_name = "sem_seg" # "drivable", "sem_seg"
config_file = "/home/zekun/drivable/src/models/config-deeplabv3plus-sem_seg.py"
checkpoint_file = "/home/zekun/drivable/outputs/semantic/models/deeplabv3+_backbone_fl10-class0-initial.pth"
train_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_train.json"
val_attr_file = f"/home/zekun/drivable/data/bdd100k/labels/{pkg_name}/bdd100k_labels_images_attributes_val.json"
output_dir = "/home/zekun/drivable/outputs/semantic"

batchsize = 2
learn_rate = 0.0001
epochs = 5
num_workers = 0

In [7]:
IMAGE_PATH, IMAGE_PATH_TRAIN, IMAGE_PATH_VAL = get_image_paths("/home/zekun/drivable/", pkg_name)
LABEL_PATH, LABEL_PATH_TRAIN, LABEL_PATH_VAL = get_label_paths("/home/zekun/drivable/", task_name, pkg_name)
MODEL_META = get_model_additional_configs(task_name)
img_transform, lbl_transform = get_transforms(task_name, output_size)
conditions = all_conditions

classes_num = MODEL_META["num_classes"]

## Data loaders

In [8]:
train_fns = tools.get_img_paths_by_conditions(conditions, train_attr_file, IMAGE_PATH_TRAIN)
val_fns = tools.get_img_paths_by_conditions(conditions, val_attr_file, IMAGE_PATH_VAL)

train_loader = get_dataloader(
    train_fns,
    batch_size=batchsize,
    workers=num_workers,
    img_transform=img_transform,
    lbl_transform=lbl_transform,
    img_path=IMAGE_PATH_TRAIN,
    lbl_path=LABEL_PATH_TRAIN,
    is_train=True,
    classes_num=classes_num
)
val_loader = get_dataloader(
    val_fns,
    batch_size=batchsize,
    workers=num_workers,
    img_transform=img_transform,
    lbl_transform=lbl_transform,
    img_path=IMAGE_PATH_VAL,
    lbl_path=LABEL_PATH_VAL,        
    is_train=False,
    classes_num=classes_num
)

## Model initialization

In [9]:
model = BDD100kModel(
    num_classes=classes_num,
    backbone=tools.load_mmcv_checkpoint(config_file, checkpoint_file),
    size=output_size
)
optimizer = optim.Adam(model.parameters(), lr=learn_rate)
criterion = nn.CrossEntropyLoss(ignore_index=255)

runner = PytorchRunner(optimizer, criterion, train_loader, val_loader, IoUMetricMeter(classes_num), DEVICE, True)

/home/zekun/.local/share/virtualenvs/drivable-kJxmeTfE/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/zekun/.local/share/virtualenvs/drivable-kJxmeTfE/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: /home/zekun/drivable/outputs/semantic/models/deeplabv3+_backbone_fl10-class0-initial.pth


## Model training

In [10]:
runner.train(model, epochs)

Train:   0%|          | 13/3484 [00:05<24:12,  2.39it/s, Loss=0.822, pAcc=0.818] 


KeyboardInterrupt: 

: 

## Model Validation

In [19]:
valid_logs = runner.validate(model)

Valid: 100%|██████████| 498/498 [00:52<00:00,  9.41it/s, Loss=0.406, pAcc=0.882]
/home/zekun/.local/share/virtualenvs/drivable-kJxmeTfE/lib/python3.10/site-packages/bdd100k/eval/seg.py:78: RuntimeWarning: invalid value encountered in divide
  ious: NDArrayF64 = np.diag(hist) / (
/home/zekun/.local/share/virtualenvs/drivable-kJxmeTfE/lib/python3.10/site-packages/bdd100k/eval/seg.py:88: RuntimeWarning: invalid value encountered in divide
  accs: NDArrayF64 = np.diag(hist) / hist.sum(axis=0)
DEBUG flames 2023-09-08 14:24:01,321 | runners.py:192 | {'name': 'validating log', 'log': {'Loss': 0.4064523779267642, 'pAcc': 0.881732966681712, 'Acc': array([0.95198948, 0.81802302, 0.89222318, 0.29443965, 0.56451934,
       0.72992805, 0.57272633, 0.72641295, 0.84147154, 0.62166514,
       0.95154021, 0.68899675, 0.        , 0.85969304, 0.20625348,
       0.79739268, 0.        , 0.        , 0.46614839]), 'mIoU': 0.4050377204942521, 'IoU': [0.8970269985059579, 0.4257580250061898, 0.7621026606385318,

{'Loss': 0.4064523779267642,
 'pAcc': 0.881732966681712,
 'Acc': array([0.95198948, 0.81802302, 0.89222318, 0.29443965, 0.56451934,
        0.72992805, 0.57272633, 0.72641295, 0.84147154, 0.62166514,
        0.95154021, 0.68899675, 0.        , 0.85969304, 0.20625348,
        0.79739268, 0.        , 0.        , 0.46614839]),
 'mIoU': 0.4050377204942521,
 'IoU': [0.8970269985059579,
  0.4257580250061898,
  0.7621026606385318,
  0.12634872245907994,
  0.2884176835367073,
  0.3578368116425738,
  0.3963294684439375,
  0.3796861744220034,
  0.800808302261701,
  0.3439375307447835,
  0.9338341766306198,
  0.4307107977346042,
  0.0,
  0.788923832711518,
  0.1538869849315081,
  0.3036525882651424,
  0.0,
  0.0,
  0.3064559314559315]}

In [61]:
for k,v in valid_logs.items():
    print(k, "=", v)

Loss = 0.31901394536147765
mAcc = 0.9128786127915733
Acc = [0.95043859 0.80964294 0.8643484  0.5441846  0.80412085 0.76782445
 0.8249825  0.82894086 0.87843651 0.55570706 0.97687499 0.86176194
 0.42660393 0.93757133 0.8047926  0.86462452 0.         0.46097626
 0.57384185]
mIoU = 0.5385917050450463
IoU = [0.9277337703984144, 0.5749089554192723, 0.8116293222447687, 0.11500538810019596, 0.42866139115677754, 0.4894062162248554, 0.5022291757045776, 0.5112494661005472, 0.8050132337244152, 0.3669570105627057, 0.9323787187552106, 0.5322123755980739, 0.324821413015071, 0.8794332374533477, 0.5546674134801459, 0.7573088192617695, 0.0, 0.2895744612381065, 0.4300520274176233]


## Appendix

In [64]:
x1 = [1,1,1]
x2 = [1,1,2]

def calculate_iou(pred_mask, true_mask, class_label):
    pred_class = pred_mask == class_label
    true_class = true_mask == class_label

    intersection = np.logical_and(pred_class, true_class)
    union = np.logical_or(pred_class, true_class)

    iou_score = np.sum(intersection) / (np.sum(union) + 0.00005)
    return iou_score

def calculate_mean_iou(pred_mask, true_mask, num_classes):
    miou_sum = 0.0
    for class_label in range(num_classes):
        iou = calculate_iou(pred_mask, true_mask, class_label)
        print(iou)
        miou_sum += iou

    mean_iou = miou_sum / num_classes
    return mean_iou

num_classes = 3
miou = calculate_mean_iou(np.array([x1, x1]), np.array([x2, x1]), num_classes)
print("mIoU:", miou)
print(x1[0])

0.0
0.8333263889467588
0.0
mIoU: 0.2777754629822529
1
